<h1> Hotel recomedation system
<h3> This system eximines the area around a location (in the example is the Excel Event center in London) then using Foursequre locates hotels within 10 KMs radius, then recommand the best to stay in based on the distance and average rating and the number of closeby resturants

In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

<h3>First we prepare credential for connecting with foursquare

In [ ]:
CLIENT_ID = 'HKWP4HM03Z0VYADRPAPOMKKB0QQMFOIFXA3SJIHEASYQWZMM' # Foursquare ID
CLIENT_SECRET = 'VDGHS2NZP1C5BIYCL0SQ30ARMDDB5XQVRD51YLRSLNESAXD4' # Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

<h3> Now we locate our central point, which is in this example the Excel even center in london

In [3]:
address = 'ExCeL London, london ,  UK'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.50814615 0.0287318446244653


<h3> We use the credential we established to request information foursequre  using the search word "hotel"

In [4]:
search_query = 'hotels'
radius = 10000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()


<h3>Next we assign the data to a dataframe and clean it up filtering unwanted columns

In [5]:
# assign relevant part of JSON to venues
hotels = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(hotels)
dataframe.shape

(30, 18)

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Hotels.com,Office,407 St John St,51.531290,-0.106847,"[{'label': 'display', 'lat': 51.53129013100273...",9738,EC1V 4EX,GB,Greater London,Greater London,United Kingdom,"[407 St John St, Greater London, EC1V 4EX, Uni...",NaN,4f869499e4b02b999b92c5b1
1,Moxy Hotel London Excel,Hotel,1014 Dockside Road,51.508510,0.041466,"[{'label': 'display', 'lat': 51.5085095, 'lng'...",883,E16 2FQ,GB,City of London,Greater London,United Kingdom,"[1014 Dockside Road, City of London, Greater L...",NaN,58c3eb9d424f933529f89a16
2,Marriott Hotels International,Building,86 Fetter Ln,51.517167,-0.109152,"[{'label': 'display', 'lat': 51.51716748003636...",9605,EC4A 1EN,GB,London,Greater London,United Kingdom,"[86 Fetter Ln, London, Greater London, EC4A 1E...",NaN,4ff19e2fe4b02f36dc3fd764
3,The Tower Hotel,Hotel,St Katherine's Way,51.506392,-0.073223,"[{'label': 'display', 'lat': 51.50639159610898...",7066,E1W 1ND,GB,London,Greater London,United Kingdom,"[St Katherine's Way, London, Greater London, E...",NaN,4b27f875f964a520098d24e3
4,Lucky 8 Hotels,Hotel,NaN,51.559290,0.068484,"[{'label': 'display', 'lat': 51.55928970584394...",6323,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,569cc2f5498e5a25c48f4794


In [ ]:
hotels_df=dataframe_filtered.drop(['categories','crossStreet','postalCode','address','formattedAddress','labeledLatLngs','cc','city', 'state', 'country',
       'formattedAddress'], axis=1)
hotels_df

In [ ]:
hotels_df.shape

<h3>To evaluate the hotels we need to acquire the rating and number of resturants around the hotel, we can achieve this by requesting more information about each hotel from foursequre
<h3> The rating can be optained by using a for loop to iterate between the differnt hotels and save the ratings
<h3> to aviod using more calls to foursequre than needed, we will first drop any hotel that is not yet rated and then use the latitude and longitude of each hotel to search for resturant in a 500 meter radius. 

In [ ]:
ids=hotels_df['id']
ratings=[]
for Id in ids:
    venue_id =Id 
    #print(Id)
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    #print (url)
    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
        ratings.append(result['response']['venue']['rating'])
    except:
        print('This venue has not been rated yet.')
        ratings.append('NA')
ratings

In [ ]:
hotels_df['rating']=ratings
hotels_df.drop(hotels_df.loc[hotels_df['rating']=='NA'].index, inplace=True)
hotels_df

In [14]:
Res_nearby= []
lat=hotels_df['lat']
lng=hotels_df['lng']
search_query = 'restaurant'
for x, y in zip(lat,lng):
    radius = 500  
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,x, y, VERSION, search_query, radius, LIMIT)
    #print(url)
    results = requests.get(url).json()
    rest = results['response']['venues']
    dataframe = json_normalize(rest)
    Res_nearby.append (len(dataframe))
Res_nearby


[3, 9, 19, 30, 20, 20, 25, 13, 8, 30]

In [ ]:
hotels_df['Nearby']=Res_nearby
hotels_df

<h3>Now we have all 3 features we want to exsimine (Distance, rating, and number of nearby resturants) of each hotel
<h3>We will create a score that is the sum of the 3 varables, but first we need to normilize them

In [17]:
hotels_df = hotels_df.reset_index(drop=True)
hotels_df

,name,lat,lng,distance,id,rating,Nearby
0,Moxy Hotel London Excel,51.508510,0.041466,883,58c3eb9d424f933529f89a16,7.2,3
1,The Tower Hotel,51.506392,-0.073223,7066,4b27f875f964a520098d24e3,7.4,9
2,Point A Hotel Shoreditch,51.522623,-0.084403,8001,58fbbe8dc21cb16c170a4cf4,7.1,19
3,Imperial Hotel,51.521697,-0.123935,10683,4b839ecdf964a5206e0b31e3,4.7,30
4,Ace Hotel Shoreditch Gym,51.525626,-0.077550,7615,52f339e8498ea23d1c2d31d8,6.8,20
5,Courthouse Hotel Shoreditch,51.527278,-0.079561,7797,5739dbf2498ede53d70af880,7.4,20
6,"Club Quarters Hotel, St. Paul's",51.513882,-0.101095,9017,4b58e941f964a5205f7228e3,6.2,25
7,The Z Hotel Shoreditch,51.526960,-0.088204,8366,55061264498e0ee5ed850341,7.4,13
8,The Bridge Hotel Southwark,51.499040,-0.101780,9100,4b6c8b0ff964a520eb412ce3,5.4,8
9,The Rathbone Hotel,51.518707,-0.135557,11442,4be52dd5d4f7c9b6b8232520,5.4,30


In [18]:
from sklearn import preprocessing
# Create x, where x the 'scores' column's values as floats
x = hotels_df[['distance']].values.astype(float)
y= hotels_df[['rating']].values.astype(float)
z= hotels_df[['Nearby']].values.astype(float)
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()
# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)
y_scaled= min_max_scaler.fit_transform(y)
z_scaled= min_max_scaler.fit_transform(z)
# Run the normalizer on the dataframe
hotels_df['distance'] = pd.DataFrame(x_scaled)
hotels_df['rating'] = pd.DataFrame(y_scaled)
hotels_df['Nearby'] = pd.DataFrame(z_scaled)

hotels_df

,name,lat,lng,distance,id,rating,Nearby
0,Moxy Hotel London Excel,51.508510,0.041466,0.000000,58c3eb9d424f933529f89a16,0.925926,0.000000
1,The Tower Hotel,51.506392,-0.073223,0.585567,4b27f875f964a520098d24e3,1.000000,0.222222
2,Point A Hotel Shoreditch,51.522623,-0.084403,0.674117,58fbbe8dc21cb16c170a4cf4,0.888889,0.592593
3,Imperial Hotel,51.521697,-0.123935,0.928118,4b839ecdf964a5206e0b31e3,0.000000,1.000000
4,Ace Hotel Shoreditch Gym,51.525626,-0.077550,0.637560,52f339e8498ea23d1c2d31d8,0.777778,0.629630
5,Courthouse Hotel Shoreditch,51.527278,-0.079561,0.654797,5739dbf2498ede53d70af880,1.000000,0.629630
6,"Club Quarters Hotel, St. Paul's",51.513882,-0.101095,0.770338,4b58e941f964a5205f7228e3,0.555556,0.814815
7,The Z Hotel Shoreditch,51.526960,-0.088204,0.708685,55061264498e0ee5ed850341,1.000000,0.370370
8,The Bridge Hotel Southwark,51.499040,-0.101780,0.778199,4b6c8b0ff964a520eb412ce3,0.259259,0.185185
9,The Rathbone Hotel,51.518707,-0.135557,1.000000,4be52dd5d4f7c9b6b8232520,0.259259,1.000000


<h3> the final score to evaluate the hotels must be a combination of all 3 variables, but we want the shortest distance and the highest rating and nearby resturants, so we will multiply the value of the rating by the normalizied value of nearby resturants and the subtract the distance value from it.
<h4>* to make the score clearer we will devide it by 2 (highest possible value) then multiply it by 10 to make more noticable

In [19]:
score=(hotels_df['rating']+hotels_df['Nearby']-hotels_df['distance'])
score=score/2*10
hotels_df['score'] = pd.DataFrame(round(score,2))


<h3>Now we can present or data and visuliaze it on a map
    <h4> we will first drop the unnesesary values

In [42]:
scores_df=hotels_df.drop(['distance','id','rating','Nearby'], axis=1)
scores_df['name']=scores_df['name'].apply(''.join).str.replace('[^A-Za-z\s]+', '') #remove special characters in hotel names for the labels in the map
scores_df=scores_df.sort_values(by=['score'],ascending=False)
scores_df=scores_df.reset_index(drop=True)
scores_df

,name,lat,lng,score
0,Courthouse Hotel Shoreditch,51.527278,-0.079561,4.87
1,Moxy Hotel London Excel,51.508510,0.041466,4.63
2,Point A Hotel Shoreditch,51.522623,-0.084403,4.04
3,Ace Hotel Shoreditch Gym,51.525626,-0.077550,3.85
4,The Z Hotel Shoreditch,51.526960,-0.088204,3.31
5,The Tower Hotel,51.506392,-0.073223,3.18
6,Club Quarters Hotel St Pauls,51.513882,-0.101095,3.00
7,The Rathbone Hotel,51.518707,-0.135557,1.30
8,Imperial Hotel,51.521697,-0.123935,0.36
9,The Bridge Hotel Southwark,51.499040,-0.101780,-1.67


In [47]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12)  # generate map centred around the Excel Center
# add the hotelss as blue circle markers
folium.Circle([latitude, longitude], 12000,color='yellow', fill = True, fill_opacity=0.25).add_child(folium.Popup('Area around Excel center')).add_to(venues_map)
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='<h4>Excel event center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.9
    ).add_to(venues_map)
for lat, lng,name,score in zip(scores_df.lat, scores_df.lng, scores_df.name,scores_df.score):
    folium.features.CircleMarker(
        [lat, lng],
        radius=score*2,
        popup=(name+'<br>'+'<h3>Score:'+str(score)+'</h3>'),
        fill = True,
        fill_color='green',
        color='green',
        fill_opacity=1
        ).add_to(venues_map)
x=scores_df['lat'][0]
y=scores_df['lng'][0]
M=scores_df['name'][0]
folium.Marker(
    location=[x, y],
    popup=M,
    icon=folium.Icon(color='yellow', icon='star')
).add_to(venues_map)
# display map
venues_map